In [1]:
import torch
import torch.nn as nn

In [2]:
embed = nn.Embedding(3,5)

In [20]:
t = torch.randint(0,3,(3,))
t

tensor([2, 1, 1])

In [22]:
embed(t)

tensor([[ 0.3009,  1.0515, -0.9211, -0.4472, -0.3552],
        [-0.3797, -0.4905, -0.0278, -0.1023, -0.8494],
        [-0.3797, -0.4905, -0.0278, -0.1023, -0.8494]],
       grad_fn=<EmbeddingBackward>)

In [23]:
embed.weight

Parameter containing:
tensor([[ 0.7972, -0.6582,  0.1128,  0.6282, -0.4751],
        [-0.3797, -0.4905, -0.0278, -0.1023, -0.8494],
        [ 0.3009,  1.0515, -0.9211, -0.4472, -0.3552]], requires_grad=True)

In [26]:
import math
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


In [28]:
pe = PositionalEncoding(6)

In [30]:
pe.pe

tensor([[[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000]],

        [[ 0.8415,  0.5403,  0.0464,  0.9989,  0.0022,  1.0000]],

        [[ 0.9093, -0.4161,  0.0927,  0.9957,  0.0043,  1.0000]],

        ...,

        [[ 0.9563, -0.2925, -0.5121,  0.8589, -0.9737, -0.2278]],

        [[ 0.2705, -0.9627, -0.4717,  0.8818, -0.9742, -0.2257]],

        [[-0.6639, -0.7478, -0.4303,  0.9027, -0.9747, -0.2236]]])

In [46]:
ohlcv = torch.tensor([
    [
        [28948.19,28997.16,28935.30,28991.01,143.661],
        [28992.98,29045.93,28991.01,29035.18,256.280],
        [29036.41,29036.97,28993.19,29016.23,102.675],
        [29016.23,29023.87,28995.50,29002.92,85.762],

    ],
    [
        [49889.01,49959.00,49700.00,49915.86,3813.991],
        [49915.87,50089.99,49823.33,49881.85,5582.806],
        [49881.85,50158.96,49830.71,50000.14,6702.779],
        [50000.92,50102.26,49921.17,50069.01,4582.944],

    ]
])
ohlcv.shape

torch.Size([2, 4, 5])

In [107]:
norm = nn.BatchNorm1d(5)
print(norm.running_mean, norm.running_var)

tensor([0., 0., 0., 0., 0.]) tensor([1., 1., 1., 1., 1.])


In [108]:
t = torch.rand(2,5)
t = torch.cat([t,t.mean(0,keepdim=True)])
t

tensor([[0.4152, 0.6089, 0.2121, 0.1288, 0.1309],
        [0.4898, 0.9148, 0.1486, 0.4143, 0.7636],
        [0.4525, 0.7619, 0.1804, 0.2716, 0.4473]])

In [109]:
norm(t)

tensor([[-1.2182, -1.2244,  1.2157, -1.2243, -1.2247],
        [ 1.2182,  1.2244, -1.2157,  1.2243,  1.2247],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       grad_fn=<NativeBatchNormBackward>)

In [110]:
print(norm.running_mean, norm.running_var)

tensor([0.0453, 0.0762, 0.0180, 0.0272, 0.0447]) tensor([0.9001, 0.9023, 0.9001, 0.9020, 0.9100])


In [115]:
print(t.mean(dim=(0)), t.std(dim=(0))*norm.momentum + torch.ones_like(t.std(dim=(0)))*(1-norm.momentum) )

tensor([0.4525, 0.7619, 0.1804, 0.2716, 0.4473]) tensor([0.9037, 0.9153, 0.9032, 0.9143, 0.9316])


In [95]:
(t - t.mean(dim=(0))*norm.momentum)/ (norm.eps + t.std(dim=(0))/norm.momentum)

tensor([[ 0.2453,  0.3311,  1.3888,  0.0214, 12.3220],
        [ 0.4453,  0.1311,  1.1888,  0.2214, 12.5219],
        [ 0.3453,  0.2311,  1.2888,  0.1214, 12.4219]])

tensor([0.0186, 0.0621, 0.0453, 0.0549, 0.0520])

In [48]:
norm(ohlcv.transpose(1,2)).transpose(2,1)

tensor([[[-1.0048, -1.0027, -1.0042, -1.0019, -0.9576],
         [-1.0005, -0.9981, -0.9988, -0.9977, -0.9147],
         [-0.9964, -0.9989, -0.9986, -0.9995, -0.9732],
         [-0.9983, -1.0002, -0.9984, -1.0008, -0.9796]],

        [[ 0.9968,  0.9887,  0.9886,  0.9951,  0.4398],
         [ 0.9994,  1.0012,  1.0004,  0.9919,  1.1132],
         [ 0.9962,  1.0077,  1.0011,  1.0032,  1.5396],
         [ 1.0075,  1.0023,  1.0098,  1.0098,  0.7325]]],
       grad_fn=<TransposeBackward0>)

In [54]:
print(ohlcv.mean(dim=(0,1)))
print(ohlcv.std(dim=(0,1)))

tensor([39460.1875, 39551.7656, 39398.7734, 39489.0234,  2658.8623])
tensor([11184.1455, 11252.6826, 11139.6514, 11201.2656,  2807.9656])


In [69]:
ohlcv

tensor([[[28948.1895, 28997.1602, 28935.3008, 28991.0098,   143.6610],
         [28992.9805, 29045.9297, 28991.0098, 29035.1797,   256.2800],
         [29036.4102, 29036.9707, 28993.1895, 29016.2305,   102.6750],
         [29016.2305, 29023.8691, 28995.5000, 29002.9199,    85.7620]],

        [[49889.0117, 49959.0000, 49700.0000, 49915.8594,  3813.9910],
         [49915.8711, 50089.9883, 49823.3281, 49881.8516,  5582.8062],
         [49881.8516, 50158.9609, 49830.7109, 50000.1406,  6702.7788],
         [50000.9219, 50102.2617, 49921.1719, 50069.0117,  4582.9438]]])

In [60]:
(ohlcv - ohlcv.mean(dim=(0,1)))/ (norm.eps + ohlcv.std(dim=(0,1)))

tensor([[[-0.9399, -0.9380, -0.9393, -0.9372, -0.8957],
         [-0.9359, -0.9336, -0.9343, -0.9333, -0.8556],
         [-0.9320, -0.9344, -0.9341, -0.9350, -0.9103],
         [-0.9338, -0.9356, -0.9339, -0.9362, -0.9164]],

        [[ 0.9325,  0.9249,  0.9247,  0.9309,  0.4114],
         [ 0.9349,  0.9365,  0.9358,  0.9278,  1.0413],
         [ 0.9318,  0.9426,  0.9365,  0.9384,  1.4402],
         [ 0.9425,  0.9376,  0.9446,  0.9445,  0.6852]]])

In [ ]:
# 시가로 나눠주고, 거래량은 batch 단위로 나눠주자